## PARTIE 2 : SAT

In [82]:
import numpy as np
import random
from itertools import combinations, chain
import subprocess

## Création d'un dataset

In [234]:
def check_class(student, w, frontiers_list, lambdou):
    """
    Retourne la classe d'un étudiant
    :param student: {np.array} Notes d'un élève donné
    :param w: {np.array} Coefficients des matières
    :param frontiers_list: {np.array[]} Frontières des notes pour chaque matière
    :param lambdou: {int} Seuil de majorité
    :return: {int} Retourne la classe d'un étudiant (ex: 0 pour refusé, 1 pour accepté)
    """
    #On part de la classe 0 et on incrémente à chaque fois que la somme des coeffs des matières que l'élève valide pour un niveau donné dépasse lambda
    student_class = 0

    #frontiers_list contient une liste de notes frontières pour chaque matières
    #on transforme pour avoir une liste de frontières des matières pour chaque classe
    frontiers_t = frontiers_list.T

    for frontiers in frontiers_t:
        coeffs_sum = ((student >= frontiers) * w).sum()
        if coeffs_sum >= lambdou:
            student_class += 1
        else:
            break

    return student_class

In [235]:
def generate_dataset(students_count, subjects_count, classes_count):
    """
    Crée un dataset de notes d'étudiant et de leur classe attribuée
    :param students_count: {int} Nombre d'étudiants
    :param subjects_count: {int} Nombre de matières
    :param classes_count:  {int} Nombre de classes (ex: 2 pour {Accepté, Refusé})
    :return: {list(np.array()[]} Liste de classes_count matrices de notes
    """
    #On génère les coefficients des différentes matières, dont la somme vaut 1
    w_ects = np.random.randint(0, 30, subjects_count)
    w = w_ects / w_ects.sum()
    print("\nLes coeffs sont = ", w)

    #Les frontières entre chaque classe (si 2 classes A et R, seulement 1 frontière)
    frontiers_list = np.zeros((subjects_count, classes_count - 1))
    for i in range(subjects_count):
        frontiers = np.sort(random.sample(list(range(1, 21)), classes_count - 1))
        frontiers_list[i,] = frontiers
    print("\nLes frontières sont = ", frontiers_list)

    #La somme des coeffs minimales à avoir pour être dans une classe donnée
    lambdou = random.uniform(0.5, 1)
    print("\nlambda = ", lambdou)

    #On génère les notes des élèves (1 ligne par élève)
    S = np.random.randint(0, 21, (students_count, subjects_count))
    print("\nLes notes des élèves :")
    print(S)

    #On regarde à quelle classe est associé chaque élève en fonction de ses notes
    y = []
    for student in S:
        student_class = check_class(student, w, frontiers_list, lambdou)
        y.append(student_class)

    print("\nLes classes des élèves :")
    print(y)

    return S, y

In [236]:
S, y = generate_dataset(10, 3, 2)


Les coeffs sont =  [0.20930233 0.13953488 0.65116279]

Les frontières sont =  [[ 9.]
 [ 8.]
 [18.]]

lambda =  0.7349990112398568

Les notes des élèves :
[[11 17 16]
 [ 6  2  5]
 [17  7  7]
 [11  1 17]
 [ 9 17  2]
 [17  8  1]
 [ 4 12  0]
 [20 10 20]
 [16  0  8]
 [ 2 18 20]]

Les classes des élèves :
[0, 0, 0, 0, 0, 0, 0, 1, 0, 1]


On génère notre dataset.

In [237]:
students_count = 4
subjects_count = 2 #les différentes matières (critères)
classes_count = 3 #Accepté ou refusé

S, y = generate_dataset(students_count, subjects_count, classes_count)


Les coeffs sont =  [0.22222222 0.77777778]

Les frontières sont =  [[ 3. 10.]
 [ 3. 10.]]

lambda =  0.6109114234076056

Les notes des élèves :
[[ 0 14]
 [ 0  1]
 [ 5  4]
 [ 3 18]]

Les classes des élèves :
[2, 0, 1, 2]


## Variables

* ### x(i, h, k)

On crée les variables binaires x(i, h, k) signifiant "la note k pour le critère i correspond à la classe k" (et qui valent donc 1 dans ce cas, 0 sinon)

In [238]:
x=[]
for student in range(students_count):
    for subject in range (subjects_count):
        for student_class in range(1, classes_count):
            x.append((subject, student_class, S[student][subject]))
            
print(x)

[(0, 1, 0), (0, 2, 0), (1, 1, 14), (1, 2, 14), (0, 1, 0), (0, 2, 0), (1, 1, 1), (1, 2, 1), (0, 1, 5), (0, 2, 5), (1, 1, 4), (1, 2, 4), (0, 1, 3), (0, 2, 3), (1, 1, 18), (1, 2, 18)]


* ### y(B) 
On crée les variables binaires y(B) signifiant "la  coalition B est suffisante" (et qui valent 1 dans ce cas, 0 sinon)

In [239]:
#Ensemble des critères
I = list(range(subjects_count))

yB = []
for n in range(0,len(I)+1):
    yB.append([i for i in combinations(I,n)])

yB=list(chain.from_iterable(yB))
print(yB)

[(), (0,), (1,), (0, 1)]


### Traduction des variables en dictionnaires pour le solveur SAT

In [240]:
x2i = {v : i+1 for i,v in enumerate(set(x))} #numérotation qui commence à 1
i2x = {i : v for v,i in x2i.items()}

print("x to Integers")
print(x2i)

print("\nIntegers to x")
print(i2x)

x to Integers
{(1, 2, 1): 1, (0, 1, 0): 2, (0, 2, 0): 3, (0, 1, 3): 4, (0, 2, 3): 5, (1, 1, 18): 6, (1, 2, 18): 7, (0, 1, 5): 8, (0, 2, 5): 9, (1, 1, 14): 10, (1, 1, 4): 11, (1, 2, 14): 12, (1, 1, 1): 13, (1, 2, 4): 14}

Integers to x
{1: (1, 2, 1), 2: (0, 1, 0), 3: (0, 2, 0), 4: (0, 1, 3), 5: (0, 2, 3), 6: (1, 1, 18), 7: (1, 2, 18), 8: (0, 1, 5), 9: (0, 2, 5), 10: (1, 1, 14), 11: (1, 1, 4), 12: (1, 2, 14), 13: (1, 1, 1), 14: (1, 2, 4)}


In [241]:
y2i = {v : i+len(x2i)+1 for i,v in enumerate(yB)} #numérotation qui commence à taille de x2i + 1
i2y = {i : v for v,i in y2i.items()}

print("yB to Integers")
print(y2i)

print("\nIntegers to yB")
print(i2y)

yB to Integers
{(): 15, (0,): 16, (1,): 17, (0, 1): 18}

Integers to yB
{15: (), 16: (0,), 17: (1,), 18: (0, 1)}


In [242]:
#Fusion des 2 dictionnaires
variables = {**x2i, **y2i}
print(variables)
i2variables = {i : v for v,i in variables.items()}
print(i2variables)

{(1, 2, 1): 1, (0, 1, 0): 2, (0, 2, 0): 3, (0, 1, 3): 4, (0, 2, 3): 5, (1, 1, 18): 6, (1, 2, 18): 7, (0, 1, 5): 8, (0, 2, 5): 9, (1, 1, 14): 10, (1, 1, 4): 11, (1, 2, 14): 12, (1, 1, 1): 13, (1, 2, 4): 14, (): 15, (0,): 16, (1,): 17, (0, 1): 18}
{1: (1, 2, 1), 2: (0, 1, 0), 3: (0, 2, 0), 4: (0, 1, 3), 5: (0, 2, 3), 6: (1, 1, 18), 7: (1, 2, 18), 8: (0, 1, 5), 9: (0, 2, 5), 10: (1, 1, 14), 11: (1, 1, 4), 12: (1, 2, 14), 13: (1, 1, 1), 14: (1, 2, 4), 15: (), 16: (0,), 17: (1,), 18: (0, 1)}


## Clauses

* ### Clause 1 : "échelle ascendente"

x(i,h,k') ∨ ¬x(i,h,k) (avec k<k')

Pour 2 notes k et k' sur un critère i, soit on a k' qui n'appartient pas à la classe h, soit k' appartient à h et dans ce cas k appartient aussi au moins à la classe h.

In [243]:
def clause1(classes_count, S, x, x2i):
    clauses_ascending_scales = []

    #On parcourt les matières
    for subject_index, subject in enumerate(S.T) : 
        #On parcourt les notes
        for student in subject:
            #On regarde les index des étudiants dont la note est inférieure
            students_inf_index = np.argwhere(subject < student)
            #Pour chaque classe
            for student_class in range (1, classes_count):
                variable_1 = x2i[(subject_index, student_class, student)]
                #Pour chaque étudiant dont la note est inférieure à l'étudiant regardé
                for index in students_inf_index:
                    variable_2 = x2i[(subject_index, student_class, subject[index[0]])]

                    if ([variable_1, -variable_2] not in clauses_ascending_scales):
                        clauses_ascending_scales.append([variable_1, -variable_2])

    return clauses_ascending_scales

clauses_ascending_scales=clause1(classes_count, S, x, x2i)
print(clauses_ascending_scales)

[[8, -2], [8, -4], [9, -3], [9, -5], [4, -2], [5, -3], [10, -13], [10, -11], [12, -1], [12, -14], [11, -13], [14, -1], [6, -10], [6, -13], [6, -11], [7, -12], [7, -1], [7, -14]]


* ### Clause 2 : "hiérarchie des profils"

x(i,h,k) ∨ ¬x(i,h',k) (avec 1<=h<h'<p-1)

Pour une note donnée k sur le critère i, soit la note appartient à la classe h, soit elle n'appartient pas à la classe h et dans ce cas elle n'appartient pas non plus à la classe du dessus.

**Remarque :** Dans le cas où on a que 2 classes (Accepté et Refusé par exemple), cet ensemble de clauses est vide car on a juste h = 1

In [244]:
def clause2(classes_count, S, x, x2i):
    clauses_hierarchy_profiles = []

    #On parcourt les classes
    for student_class in range(1, classes_count):
        #on regarde les classes qui sont inférieures :
        classes_inf = list(range(1, student_class))

        #on parcourt les matières:
        for subject_index, subject in enumerate(S.T) :
            #On parcourt les notes
            for student in subject :
                variable_1 = x2i[(subject_index, student_class, student)]
                #on parcourt les classes qui sont inférieures
                for classe_inf in classes_inf :
                    variable_2 = x2i[(subject_index, classe_inf, student)]

                    if ([variable_2, -variable_1] not in clauses_hierarchy_profiles):
                        clauses_hierarchy_profiles.append([variable_2, -variable_1])

    return(clauses_hierarchy_profiles)
     
clauses_hierarchy_profiles=clause2(classes_count, S, x, x2i)
print(clauses_hierarchy_profiles)

[[2, -3], [8, -9], [4, -5], [10, -12], [13, -1], [11, -14], [6, -7]]


* ### Clause 3 : "force de coalitions"

yB' ∨ ¬yB (avec B ⊂ B' ⊆ N )

Soit la coalition B n'est pas suffisante, soit elle l'est et dans ce cas B' l'est aussi.

In [245]:
def clause3(yB, y2i):
    clauses_coalition_strength = []

    #On parcourt les différentes coalitions
    for index, coalition in enumerate(yB):
        variable_1 = y2i[coalition]
        #On parcourt les coalitions plus petites
        coalitions_inf=[]
        for i in range(index):
            #si elle est inclue dans notre coalition
            if set(yB[i]).issubset(coalition):
                coalitions_inf.append(yB[i])

        for coalition_inf in coalitions_inf:
            variable_2 = y2i[coalition_inf]

            if ([variable_1, -variable_2] not in clauses_coalition_strength):
                clauses_coalition_strength.append([variable_1, -variable_2])
                
    return clauses_coalition_strength


clauses_coalition_strength=clause3(yB, y2i)
print(clauses_coalition_strength)

[[16, -15], [17, -15], [18, -15], [18, -16], [18, -17]]


* ### Clause 4 :

Vi∈B(¬x(i,h,ui)) ∨ ¬yB

Si toutes les notes de u (qui a été évalué par la classe h-1) sont au-dessus de la frontière de la classe h, alors B n'est pas une coalition suffisante.

Autre formulation : Soit la coalition B n'est pas suffisante, soit elle l'est et dans ce cas il y a au moins une note de u (évalué par la classe h-1) qui ne peut pas être de la classe h.

In [246]:
def clause4(classes_count, S, yB, y2i, x2i):
    clauses4= []

    #On parcourt les coalitions
    for coalition in yB:
        variable_1 = y2i[coalition]

        #On parcourt les étudiants
        for student in S:
            variables_list = [-variable_1]

            #On parcourt les classes:
            for student_class in range(1, classes_count):
                if student_class < classes_count -1:
                    class_sup = student_class+1

                    #On parcourt les matières
                    for subject_index, grade in enumerate(student) :
                        variable_2 = x2i[(subject_index, class_sup, grade)]
                        variables_list.append(-variable_2)

                if variables_list not in clauses4:
                    clauses4.append(variables_list)

    return clauses4

clauses4 = clause4(classes_count, S, yB, y2i, x2i)
print(clauses4)          

[[-15, -3, -12], [-15, -3, -1], [-15, -9, -14], [-15, -5, -7], [-16, -3, -12], [-16, -3, -1], [-16, -9, -14], [-16, -5, -7], [-17, -3, -12], [-17, -3, -1], [-17, -9, -14], [-17, -5, -7], [-18, -3, -12], [-18, -3, -1], [-18, -9, -14], [-18, -5, -7]]


* ### Clause 5 :

Vi∈B(x(i,h,ai)) ∨ yN\B

In [248]:
def clause5(classes_count, S, yB, y2i, x2i):
    clauses5= []

    #On parcourt les coalitions
    for coalition in yB:
        
        #Le complémentaire de cette coalition :
        coalition_comp = [elem for elem in yB if set(elem).isdisjoint(coalition) and set((*elem,*coalition)).issuperset(set(range(subjects_count)))][0]
        
        variable_1 = y2i[coalition_comp]

        #On parcourt les étudiants
        for student in S:
            variables_list = [-variable_1]

            #On parcourt les classes:
            for student_class in range(1, classes_count):
                if student_class < classes_count -1:
                    class_sup = student_class+1

                    #On parcourt les matières
                    for subject_index, grade in enumerate(student) :
                        variable_2 = x2i[(subject_index, class_sup, grade)]
                        variables_list.append(variable_2)

                if variables_list not in clauses5:
                    clauses5.append(variables_list)

    return clauses5

clauses5 = clause5(classes_count, S, yB, y2i, x2i)
print(clauses5)          

[[-18, 3, 12], [-18, 3, 1], [-18, 9, 14], [-18, 5, 7], [-17, 3, 12], [-17, 3, 1], [-17, 9, 14], [-17, 5, 7], [-16, 3, 12], [-16, 3, 1], [-16, 9, 14], [-16, 5, 7], [-15, 3, 12], [-15, 3, 1], [-15, 9, 14], [-15, 5, 7]]


## Construction du DIMACS et résolution

In [249]:
def clauses_to_dimacs(clauses,numvar) :
    dimacs = 'c This is it\np cnf '+str(numvar)+' '+str(len(clauses))+'\n'
    for clause in clauses :
        for atom in clause :
            dimacs += str(atom) +' '
        dimacs += '0\n'
    return dimacs

def write_dimacs_file(dimacs, filename):
    with open(filename, "w", newline="") as cnf:
        cnf.write(dimacs)

#Attention à utiliser la vesion du solveur compatible avec votre système d'exploitation, mettre le solveur dans le même dossier que ce notebook        

def exec_gophersat(filename, cmd = "./NCS/Exemple_SAT_Test/gophersat/win64/gophersat-1.1.6.exe", encoding = "utf8") :
    result = subprocess.run([cmd, filename], stdout=subprocess.PIPE, check=True, encoding=encoding)
    string = str(result.stdout)
    lines = string.splitlines()
    
    print(lines)

    if lines[1] != "s SATISFIABLE":
        return False, [], {}

    model = lines[2][2:].split(" ")
    
    return True, [int(x) for x in model[:-1] if int(x) != 0], { i2variables[abs(int(v))] : int(v) > 0 for v in model if int(v)!=0} 


In [250]:
#Lancer la résolution

myClauses= clauses_ascending_scales + clauses_hierarchy_profiles + clauses_coalition_strength + clauses4 + clauses5
myDimacs = clauses_to_dimacs(myClauses,len(variables))

write_dimacs_file(myDimacs,"workingfile.cnf")
res = exec_gophersat("workingfile.cnf")

#Résultat
print(res)

['c solving workingfile.cnf', 's SATISFIABLE', 'v -1 -2 -3 -4 -5 -6 -7 -8 -9 -10 -11 -12 -13 -14 -15 -16 -17 -18 0']
(True, [-1, -2, -3, -4, -5, -6, -7, -8, -9, -10, -11, -12, -13, -14, -15, -16, -17, -18], {(1, 2, 1): False, (0, 1, 0): False, (0, 2, 0): False, (0, 1, 3): False, (0, 2, 3): False, (1, 1, 18): False, (1, 2, 18): False, (0, 1, 5): False, (0, 2, 5): False, (1, 1, 14): False, (1, 1, 4): False, (1, 2, 14): False, (1, 1, 1): False, (1, 2, 4): False, (): False, (0,): False, (1,): False, (0, 1): False})
